## 통계 기반 기법 개선하기

## PMI : 점별 상호 정보량(Pointwise Mutual Information), log 함수
#### 높을수록 관련성이 높다

![image](https://t1.daumcdn.net/cfile/tistory/267AA340587EFB8737)

### PPMI(Positive PMI) : 양의 상호 정보량,  PPMI(x,y) = max(0,PMI(x,y))

In [1]:
from mynlp import preprocess, create_co_matrix, cos_similarity, most_similar
import numpy as np

In [3]:
text = 'You say goodbye and I say hello.' 
corpus, word_to_id, id_to_word = preprocess(text)
print(corpus)
print(word_to_id)
vocab_size = len(word_to_id)  # 7
C = create_co_matrix(corpus,vocab_size)
print('동시발생행렬:\n',C)
C.shape

[0 1 2 3 4 1 5 6]
{'you': 0, 'say': 1, 'goodbye': 2, 'and': 3, 'i': 4, 'hello': 5, '.': 6}
동시발생행렬:
 [[0 1 0 0 0 0 0]
 [1 0 1 0 1 1 0]
 [0 1 0 1 0 0 0]
 [0 0 1 0 1 0 0]
 [0 1 0 1 0 0 0]
 [0 1 0 0 0 0 1]
 [0 0 0 0 0 1 0]]


(7, 7)

### PPMI 행렬

In [25]:
M =  np.zeros_like(C, dtype=np.float32) # (7,7)

N = np.sum(C)  # 14 , 동시발생행렬 요소전체의 합
print(N)

S = np.sum(C,axis=0) # 동시발생행렬 요소의 수직 방향 합
print(S)             # [1 4 2 2 2 2 1]

total = C.shape[0]*C.shape[1]  # 49, 동시발생행렬 요소의 총갯수

for i in range(C.shape[0]) : # 7 회
    for j in range(C.shape[1]) : # 7 회
        # C[i,j] : 동시발생행렬의 요소한개의 값, 비교하려는 두 단어의 동시발생횟수
        # S[0] : 동시발생행렬에서 열의 'you' 단어의 동시 발생횟수
        # S[i] : 동시발생행렬에서 열의 한 단어의 동시 발생횟수 , 비교하려는 두단어 중 하나 ,x
        # S[j] : 동시발생행렬에서 열 한 단어에 대해 비교하려는 나머지 단어들의 동시 발생횟수, y
        # print(i,j,C[i,j])
        # print('S[%d]:'%i,S[i],'S[%d]:'%j,S[j])
        
        pmi = np.log2(C[i,j]*N /(S[i]*S[j]) + 1e-8)
        M[i,j] = max(0,pmi)
        
print('PPMI:\n', M)   # PPMI 행렬로 변환 
print(C)

14
[1 4 2 2 2 2 1]
PPMI:
 [[0.        1.8073549 0.        0.        0.        0.        0.       ]
 [1.8073549 0.        0.8073549 0.        0.8073549 0.8073549 0.       ]
 [0.        0.8073549 0.        1.8073549 0.        0.        0.       ]
 [0.        0.        1.8073549 0.        1.8073549 0.        0.       ]
 [0.        0.8073549 0.        1.8073549 0.        0.        0.       ]
 [0.        0.8073549 0.        0.        0.        0.        2.807355 ]
 [0.        0.        0.        0.        0.        2.807355  0.       ]]
[[0 1 0 0 0 0 0]
 [1 0 1 0 1 1 0]
 [0 1 0 1 0 0 0]
 [0 0 1 0 1 0 0]
 [0 1 0 1 0 0 0]
 [0 1 0 0 0 0 1]
 [0 0 0 0 0 1 0]]


In [26]:
# PPMI 행렬 변환 함수 구현 
def ppmi(C, verbose=False, eps=1e-8):
    M =  np.zeros_like(C, dtype=np.float32) 
    N = np.sum(C)  
    S = np.sum(C,axis=0)
    total = C.shape[0]*C.shape[1]  
    
    cnt = 0

    for i in range(C.shape[0]) : 
        for j in range(C.shape[1]) :
            pmi = np.log2(C[i,j]*N /(S[i]*S[j]) + eps)
            M[i,j] = max(0,pmi)
            
            if verbose:
                cnt += 1
                if cnt % (total//100) == 0 : # total 이 100 보다 클 때만 사용
                    print('%.1f%% 완료'% (100*cnt/total))            
    return M 

In [29]:
# total = 49
# print(total//100)
# cnt = 1
# cnt % (total//100)

In [33]:
text = 'You say goodbye and I say hello.' 
corpus, word_to_id, id_to_word = preprocess(text)
# print(corpus)
# print(word_to_id)
vocab_size = len(word_to_id)  # 7
C = create_co_matrix(corpus,vocab_size)

W = ppmi(C)
print(W)   # 동시발생행렬보다 좀더 나은 확률에 기반한 PPMI행렬로 변환되었다

[[0.        1.8073549 0.        0.        0.        0.        0.       ]
 [1.8073549 0.        0.8073549 0.        0.8073549 0.8073549 0.       ]
 [0.        0.8073549 0.        1.8073549 0.        0.        0.       ]
 [0.        0.        1.8073549 0.        1.8073549 0.        0.       ]
 [0.        0.8073549 0.        1.8073549 0.        0.        0.       ]
 [0.        0.8073549 0.        0.        0.        0.        2.807355 ]
 [0.        0.        0.        0.        0.        2.807355  0.       ]]


### SVD(Singular Value Decomposition, 특잇값분해) 에 의한 차원의 축소

#### 차원 축소 알고리즘 정리
- PCA(Principal Component Analysis,주성분 분석) : 데이터의 분산(variance)을 최대한 보존하면서 서로 직교하는 새 기저(축)를 찾아, 고차원 공간의 표본들을 선형 연관성이 없는 저차원 공간으로 변환하는 기법, 정방행렬인 공분산행렬(covariance matrix)을 고유벡터(eigenvector)로 분해한다

 https://ratsgo.github.io/machine%20learning/2017/04/24/PCA/ 
<br>

- SVD(Singular Value Decomposition, 특잇값분해) : 임의의 행렬을 세 행렬의 곱으로 분해

$$ X=USV^T $$
      
   https://angeloyeo.github.io/2019/08/01/SVD.html
  
   https://darkpgmr.tistory.com/106 

[노트북문서화작업]

http://www.onemathematicalcat.org/MathJaxDocumentation/MathJaxKorean/TeXSyntax_ko.html

In [35]:
print(W)

[[0.        1.8073549 0.        0.        0.        0.        0.       ]
 [1.8073549 0.        0.8073549 0.        0.8073549 0.8073549 0.       ]
 [0.        0.8073549 0.        1.8073549 0.        0.        0.       ]
 [0.        0.        1.8073549 0.        1.8073549 0.        0.       ]
 [0.        0.8073549 0.        1.8073549 0.        0.        0.       ]
 [0.        0.8073549 0.        0.        0.        0.        2.807355 ]
 [0.        0.        0.        0.        0.        2.807355  0.       ]]


In [37]:
U,S,V = np.linalg.svd(W)   #  PPMI 행렬 W을 밀집벡터 U로 변환

np.set_printoptions(precision=2)
print(U)

[[-3.41e-01 -1.11e-16 -3.89e-16 -1.21e-01  0.00e+00  9.32e-01  2.23e-16]
 [ 0.00e+00 -5.98e-01  1.80e-01  0.00e+00 -7.81e-01  0.00e+00  0.00e+00]
 [-4.36e-01 -4.24e-17 -2.17e-16 -5.09e-01 -1.77e-17 -2.25e-01 -7.07e-01]
 [-2.61e-16 -4.98e-01  6.80e-01 -6.57e-17  5.38e-01  9.95e-17  1.20e-17]
 [-4.36e-01 -3.23e-17 -1.65e-16 -5.09e-01 -1.35e-17 -2.25e-01  7.07e-01]
 [-7.09e-01 -3.23e-17 -1.65e-16  6.84e-01 -1.35e-17 -1.71e-01  5.89e-17]
 [ 3.06e-16 -6.28e-01 -7.10e-01  8.85e-17  3.17e-01 -2.85e-16 -1.55e-17]]


In [38]:
# 차원을 줄이려면 처음 2개의 요소를 꺼내면 된다
print(U[:,:2])    # (7,7)  --> (7,2) 로 축소됨

[[-3.41e-01 -1.11e-16]
 [ 0.00e+00 -5.98e-01]
 [-4.36e-01 -4.24e-17]
 [-2.61e-16 -4.98e-01]
 [-4.36e-01 -3.23e-17]
 [-7.09e-01 -3.23e-17]
 [ 3.06e-16 -6.28e-01]]
